In [1]:
from multi_arm import MultiArmedBanditEnv

env: MultiArmedBanditEnv = MultiArmedBanditEnv(n_arms=5, seed=42)
obs: int = env.reset()
done: bool = False
total_reward: int = 0

print('Multi-armed state', env.probabilities)
while not done:
    action: int = env.action_space.sample()  # Replace this with your strategy
    obs, reward, done, info = env.step(action)
    total_reward += reward
    print(f"Action: {action}, Reward: {reward}, Total Reward: {total_reward}")

env.close()

Multi-armed state [0.77395605 0.43887844 0.85859792 0.69736803 0.09417735]
Action: 1, Reward: 0, Total Reward: 0
Action: 0, Reward: 1, Total Reward: 1
Action: 1, Reward: 0, Total Reward: 1
Action: 1, Reward: 1, Total Reward: 2
Action: 1, Reward: 0, Total Reward: 2
Action: 1, Reward: 1, Total Reward: 3
Action: 1, Reward: 0, Total Reward: 3
Action: 0, Reward: 1, Total Reward: 4
Action: 2, Reward: 1, Total Reward: 5
Action: 2, Reward: 1, Total Reward: 6
Action: 3, Reward: 1, Total Reward: 7
Action: 3, Reward: 1, Total Reward: 8
Action: 3, Reward: 1, Total Reward: 9
Action: 1, Reward: 0, Total Reward: 9
Action: 4, Reward: 0, Total Reward: 9
Action: 4, Reward: 0, Total Reward: 9
Action: 2, Reward: 1, Total Reward: 10
Action: 4, Reward: 0, Total Reward: 10
Action: 0, Reward: 0, Total Reward: 10
Action: 3, Reward: 0, Total Reward: 10
Action: 3, Reward: 1, Total Reward: 11
Action: 1, Reward: 0, Total Reward: 11
Action: 3, Reward: 1, Total Reward: 12
Action: 0, Reward: 1, Total Reward: 13
Actio

In [9]:
import numpy as np

class ExploreAndGreed:
    def __init__(self, arm_num: int =1, eploration_prob: float =0.1) -> None:
        assert 0 <= eploration_prob <= 1, f'"eploration_prob" should be in [0, 1] range, current value = {eploration_prob}'
        self.e = eploration_prob
        self.arm_num = arm_num
        self.arm_count = np.zeros(arm_num)
        self.arm_reward = np.zeros(arm_num) 
    
    def act(self) -> int:
        # act based on current internal state

        # exploitation step
        if  np.random.random() > self.e:
            return np.argmax(self.arm_reward / (self.arm_count + 1e-5))
        else: # exploration step
            return np.random.choice(self.arm_num)
    
    def update_state(self, action: int, reward: int) -> None:
        # update stats based on observed reward
        self.arm_count[action] += 1
        self.arm_reward[action] += reward


In [10]:
print('Multi-armed state', env.probabilities)
agent = ExploreAndGreed(env.n_arms)
obs: int = env.reset()
done: bool = False
total_reward: int = 0
while not done:
    action: int = agent.act()  # Replace this with your strategy
    obs, reward, done, info = env.step(action)
    agent.update_state(action, reward)
    total_reward += reward
    print(f"Action: {action}, Reward: {reward}, Total Reward: {total_reward}")

Multi-armed state [0.77395605 0.43887844 0.85859792 0.69736803 0.09417735]
Action: 0, Reward: 0, Total Reward: 0
Action: 0, Reward: 1, Total Reward: 1
Action: 0, Reward: 1, Total Reward: 2
Action: 0, Reward: 0, Total Reward: 2
Action: 0, Reward: 1, Total Reward: 3
Action: 0, Reward: 1, Total Reward: 4
Action: 0, Reward: 1, Total Reward: 5
Action: 0, Reward: 1, Total Reward: 6
Action: 0, Reward: 1, Total Reward: 7
Action: 0, Reward: 1, Total Reward: 8
Action: 2, Reward: 1, Total Reward: 9
Action: 2, Reward: 1, Total Reward: 10
Action: 2, Reward: 1, Total Reward: 11
Action: 2, Reward: 1, Total Reward: 12
Action: 2, Reward: 0, Total Reward: 12
Action: 0, Reward: 1, Total Reward: 13
Action: 0, Reward: 0, Total Reward: 13
Action: 2, Reward: 1, Total Reward: 14
Action: 2, Reward: 1, Total Reward: 15
Action: 2, Reward: 1, Total Reward: 16
Action: 2, Reward: 0, Total Reward: 16
Action: 2, Reward: 0, Total Reward: 16
Action: 0, Reward: 1, Total Reward: 17
Action: 0, Reward: 1, Total Reward: 18


In [11]:
agent.arm_reward

array([ 613.,   94., 7337.,  141.,   19.])